### インポート

In [ ]:
#一度pip install したらコメントアウトしてください!!
# !pip install beautifulsoup4
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import requests
import time
from bs4 import BeautifulSoup
import os

### アベレージヒッターのOPS

In [ ]:
# データの読み込みと前処理 (エンコーディング修正済み)
# 想定される2つのパスをリストアップ
# パターン1: work_datasetフォルダ内
path_pattern_1 = 'work_dataset/stats_bat.csv'

# パターン2:実行ファイルと同じ階層
path_pattern_2 = 'stats_bat.csv'

# どちらのパスを使うか決定するための変数
use_file_path = None

# --- ファイルの存在チェック ---

if os.path.exists(path_pattern_1):
    use_file_path = path_pattern_1
    print(f"パターン1のパスを採用しました: {use_file_path}")

elif os.path.exists(path_pattern_2):
    use_file_path = path_pattern_2
    print(f"パターン2のパスを採用しました: {use_file_path}")

else:
    # どちらも見つからなかった場合
    print("エラー: 'stats_bat.csv' が見つかりません。")
    print(f"以下の場所を確認してください:")
    print(f" - {path_pattern_1}")
    print(f" - {path_pattern_2}")
    # プログラムをここで終了するか、処理をスキップする
    use_file_path = None


# --- 読み込み処理 ---

if use_file_path:
    try:
        # 見つかった方のパスで読み込み実行
        df = pd.read_csv(use_file_path, encoding='cp932')
        print("データフレームの読み込みに成功しました。")
        # print(df.head())

    except Exception as e:
        print(f"データの読み込み中にエラーが発生しました: {e}")

file_path = use_file_path
db_name = 'baseball.db'

try:
    df = pd.read_csv(file_path, encoding='cp932')

    with sqlite3.connect(db_name) as conn:
        df.to_sql('players', conn, if_exists='replace', index=False)
        cur = conn.cursor()

        sql_select = '''
            SELECT 
                season_year, 
                team_name,    -- ★追加: チーム名
                player_name, 
                g, 
                (h * 1.0 / ab) as ba,     -- 打率
                hr, 
                (tb * 1.0 / ab) as slg,   -- 長打率
                
                -- OPSの計算
                (
                  ((h + bb + hbp) * 1.0 / (ab + bb + hbp + sf)) 
                  + 
                  (tb * 1.0 / ab)
                ) as ops

            FROM players 
            WHERE 
                  ab > 0 
              AND (h * 1.0 / ab) >= 0.250
              AND hr <= 9
              AND g >= 100
              AND season_year NOT IN (2021, 2022)
            
            ORDER BY season_year ASC, ops DESC
        '''
        
        cur.execute(sql_select)
        results = cur.fetchall()

        print("-" * 100)
        print(f"条件一致: {len(results)}件")
        print("-" * 100)
        
        for row in results:

            print(f"{row[0]}年 | {row[1]:<6} | {row[2]:<6} | {row[3]}試合 | 打率:{row[4]:.3f} | 本塁打:{row[5]} | 長打率:{row[6]:.3f} | OPS:{row[7]:.3f}")

except Exception as e:
    print(f"エラー: {e}")

In [ ]:
plt.rcParams['font.family'] = 'Hiragino Sans'

file_path = 'stats_bat.csv'
db_name = 'baseball.db'

try:
    with sqlite3.connect(db_name) as conn:
        # 分析用のデータを取得
        sql_select = '''
            SELECT 
                player_name, 
                hr, 
                (
                  ((h + bb + hbp) * 1.0 / (ab + bb + hbp + sf)) + (tb * 1.0 / ab)
                ) as ops
            FROM players 
            WHERE ab > 0 
              AND (h * 1.0 / ab) >= 0.250
              AND hr <= 9
              AND g >= 100
              AND season_year NOT IN (2021, 2022)
        '''
        df_graph = pd.read_sql(sql_select, conn)

        # 散布図の作成
        plt.figure(figsize=(10, 6))
        
        # 散布図プロット
        plt.scatter(df_graph['hr'], df_graph['ops'], alpha=0.6, color='blue')

        # 平均線の描画
        plt.axhline(y=0.700, color='gray', linestyle='--' )
        plt.axhline(y=0.750, color='red', linestyle='--', label='優秀な打者 (.750)')

        # OPSが高い上位の選手名を表示する
        for i, row in df_graph.iterrows():
            if row['ops'] >= 0.750:
                # 名前を表示
                plt.text(row['hr'], row['ops'] + 0.002, row['player_name'], fontsize=9)

        plt.title('「本塁打9本以下」選手の貢献度分析 (OPS分布)', fontsize=14)
        plt.xlabel('本塁打数', fontsize=12)
        plt.ylabel('OPS (出塁率+長打率)', fontsize=12)
        plt.grid(True)
        plt.legend()
        
        # グラフ表示
        plt.show()

except Exception as e:
    print(f"エラー: {e}")

### 考察

In [8]:
#   優秀な選手のOPS平均目安は.750以上
#   小園海斗、泉口友太、柳町樹、福永祐基、辰巳涼介、岡大海、蝦名達夫、近本光司、森友哉
#   順位が上位の選手がずらりと並んでいることから、チームへの貢献度が高いのではないかと思った